![Header](imgs/header.jpg)

### Previsão de tempo de duração de uma viagem de Taxi de Nova York

Francisco Igor da Silva Lima - 507536

O desafio [New York City Taxi Trip Duration](https://www.kaggle.com/c/nyc-taxi-trip-duration/) do [Kaggle](https://www.kaggle.com/) é uma desafio de regressão que tem como finalidade prever a duração de uma viagem de taxi na cidade de Nova Yorque.

Iremos realizar a regressão utilizando uma Regressão Linear e Support Vector Regression

In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from datetime import datetime

In [2]:
# import dos dados

df = pd.read_csv('train_taxi.csv')

In [3]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


## Pre processamento

### Significados dos Campos:

- id - a unique identifier for each trip
- vendor_id - a code indicating the provider associated with the trip record
- pickup_datetime - date and time when the meter was engaged
- dropoff_datetime - date and time when the meter was disengaged
- passenger_count - the number of passengers in the vehicle (driver entered value)
- pickup_longitude - the longitude where the meter was engaged
- pickup_latitude - the latitude where the meter was engaged
- dropoff_longitude - the longitude where the meter was disengaged
- dropoff_latitude - the latitude where the meter was disengaged
- store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
- trip_duration - duration of the trip in seconds


In [4]:
df = df.drop(['id', 'dropoff_datetime', 'store_and_fwd_flag'], axis=1)

In [5]:
def time_to_timestamp(value):
    return value.hour *60 + value.day

In [6]:
# transformar o pickup_datetime em valor numérico
df['pickup_datetime'] = df['pickup_datetime'].apply(lambda x:  time_to_timestamp(datetime.strptime(x, '%Y-%m-%d %H:%M:%S')))

In [7]:
df.head()

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,2,1034,1,-73.982155,40.767937,-73.964630,40.765602,455
1,1,12,1,-73.980415,40.738564,-73.999481,40.731152,663
2,2,679,1,-73.979027,40.763939,-74.005333,40.710087,2124
3,2,1146,1,-74.010040,40.719971,-74.012268,40.706718,429
4,2,806,1,-73.973053,40.793209,-73.972923,40.782520,435


In [8]:
df.shape

(1458644, 8)

### Divisão do Dataset

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df[['pickup_datetime', 'pickup_longitude', 'pickup_latitude',
        'dropoff_longitude', 'dropoff_latitude', 'vendor_id', 'passenger_count']]
y = df['trip_duration']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.333, random_state=42)

In [13]:
print(df.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_valid.shape)
print(y_valid.shape)

(1458644, 8)
(1021050, 7)
(1021050,)
(291875, 7)
(291875,)
(145719, 7)
(145719,)


In [14]:
X_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id,passenger_count
1011257,543,-73.979813,40.752529,-73.982681,40.763680,2,2
178550,961,-73.977623,40.746296,-73.968414,40.788933,1,1
551460,804,-73.950699,40.779404,-73.863129,40.768608,1,1
1434271,1030,-73.984558,40.721802,-73.976753,40.752213,2,1
1173610,1045,-73.967720,40.768742,-73.977173,40.789875,1,1


### Treinamento

Iremos começar pela Regressão Linear

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error

In [16]:
lr = LinearRegression().fit(X_train, y_train)

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [18]:
y_predicted = lr.predict(X_test)

In [19]:
y_predicted.shape

(291875,)

In [20]:
y_predicted.min()

-9694.7181619053

In [21]:
# a métrica que iremos utilizar necessita que todos os valores 
#    sejam maiores que zero, por iremos fazer essa conversão
#    até pq não existem minutos negativos, o que é um erro
#    do modelo, uma viagem não pode durar -8 minutos, por exemplo
y_predicted = np.array([i if i>=0 else 0 for i in y_predicted])

In [22]:
y_predicted.min()

0.0

A métrica que iremos utilizar é o RMSLE (Erro logarítmico médio quadrático) que é a métrica utilizada na competição.


$$
\epsilon = \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 }
$$

$\epsilon$  é o score RMSLE )

$n$ é o número de observações no dataset

$p_i$ é o valor predito para \\(i\\)

$a_i$ é o valor real para \\(i\\)

$\log(x)$ é o logaritmo natural de  \\(x\\)

In [23]:
print(f"O RMSLE obtido foi de: {np.sqrt(mean_squared_log_error(y_predicted, y_test))}")

O RMSLE obtido foi de: 0.868642688411222


In [24]:
#df = df.drop(['id', 'dropoff_datetime', 'store_and_fwd_flag'], axis=1)

### Standardização

Vamos standizar os dados para tentar melhorar as predições

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [27]:
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [28]:
lr = LinearRegression()
lr.fit(X_train_std, y_train)

LinearRegression()

In [29]:
y_predicted = lr.predict(X_test_std)

In [30]:
y_predicted.shape

(291875,)

In [31]:
y_predicted.min()

-9694.718161905545

In [32]:
# semelhante ao modelo passado precisamos realizar o mesmo processamento
y_predicted_std = np.array([i if i>=0 else 0 for i in y_predicted])

In [33]:
y_predicted_std.min()

0.0

In [34]:
print(f"O RMSLE obtido foi de: {np.sqrt(mean_squared_log_error(y_predicted_std, y_test))}")

O RMSLE obtido foi de: 0.8686426884112426


In [35]:
#del df, X, y, X_train, X_test, y_train, y_test, y_predicted

Podemos concluir que o uma Regressão Linear não é um bom modelo para esse modelo

### Arvore de Decisão

Uma árvore de decisão tenta tomar decisões baseado em regras que a própria árvore infere sobre os dados.

A árvore tem nós e folhas, nos nós temos as "regras" e nas folhas temos os valores da decisão. Tenhamos como exemplo o problema do [titanic](https://www.kaggle.com/c/titanic), em um nó, podemos ter variável sexo == HOMEM as pessoas que não mulheres podem ser classificadas sobreviventes.



![Header](imgs/titanic-decision-tree.png)


In [36]:
from sklearn.tree import DecisionTreeRegressor

In [37]:
regressor = DecisionTreeRegressor(random_state=42)

In [38]:
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [39]:
y_pred = regressor.predict(X_test)

In [40]:
print(f"O RMSLE obtido foi de: {np.sqrt(mean_squared_log_error(y_pred, y_test))}")

O RMSLE obtido foi de: 0.6169452365405642


O resultado obtido com uma Arvore de Decisão foi bastante superior ao da Regressão Linear, mas mesmo assim vamos treintar com outros valores

In [41]:
regressor = DecisionTreeRegressor(random_state=42)

In [42]:
regressor.fit(X_train_std, y_train)

DecisionTreeRegressor(random_state=42)

In [43]:
y_pred_std = regressor.predict(X_test_std)

In [44]:
print(f"O RMSLE obtido foi de: {np.sqrt(mean_squared_log_error(y_pred_std, y_test))}")

O RMSLE obtido foi de: 0.6168712048250917


In [45]:
# o resultado obtido foi praticamente o mesmo

![Header](imgs/taxi.gif)
